T VS F CLASSIFICATION

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

binary_data_path = '/content/drive/MyDrive/NLP_Workspace/Data/BERT_data/'

df = pd.read_csv(binary_data_path + "F_vs_T_data.csv")
df = df.dropna(how='any',axis=0)
df.sample(10)

,label,text
272036,1,"I'd say Yoshiki is ISTP, and I definitely agre..."
9871,1,I wish more people would vote but like actual...
257196,1,"Well if you don't change, that's great. I advi..."
51549,1,"Strangely enough, I actually have a notebook t..."
178370,1,"'No, no, and no. In my opinion. I believe the ..."
417069,1,[' ']
176051,1,"Okay, truth time. I'm not actually studying y..."
25034,1,"Good advice, it's crazy how much you learn whe..."
217396,0,'Thinking Fast and Slow. It helped me understa...
69380,0,A serious lack of empathy across the board.


In [ ]:
DOWNSAMPLE = 30000
df_balanced = df.sample(DOWNSAMPLE)
df_balanced['label'].value_counts()

1    16807
0    13193
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_balanced, test_size=0.15)

In [ ]:
train.head(3)

,label,text
34262,0,double post.
257641,1,Pagan is goooooood. And I will never stop mono...
195172,1,This is funny: How Not To Draw: Anatomy of a ...


In [ ]:
test.head(3)

,label,text
209789,0,"This is the most asked question in the forum, ..."
7104,1,Actually freedom sounds more like this one
284052,1,"I'm sorry, I don't mean to offend you. But st..."


In [ ]:
ourpur_dir = binary_data_path + "TvsF_BERT/"
train.to_csv(ourpur_dir+"T_vs_F_train_data.csv", index=False)
test.to_csv(ourpur_dir+"T_vs_F_test_data.csv", index=False)

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 8.1 MB/s 
     |████████████████████████████████| 182 kB 78.4 MB/s 
     |████████████████████████████████| 212 kB 67.7 MB/s 
     |████████████████████████████████| 132 kB 77.9 MB/s 
     |████████████████████████████████| 127 kB 74.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
data_files = {"train": ourpur_dir+"T_vs_F_train_data.csv", "test": ourpur_dir+"T_vs_F_test_data.csv"}

from datasets import load_dataset
squad_it_dataset = load_dataset("csv", data_files=data_files, sep=',')



Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8172bacb60f525c9/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 25500
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 4500
    })
})

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 7.1 MB/s 
     |████████████████████████████████| 7.6 MB 46.9 MB/s 


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

In [ ]:
TRAIN_NUM = int(DOWNSAMPLE * 0.85)
TEST_NUM = int(DOWNSAMPLE * 0.15)
TEST_NUM

4500

In [ ]:
train_dataset = squad_it_dataset["train"].shuffle(seed=42).select([i for i in list(range(TRAIN_NUM))])
test_dataset = squad_it_dataset["test"].shuffle(seed=42).select([i for i in list(range(TEST_NUM))])

tokenized_train = train_dataset.map(preprocess_function)
tokenized_test = test_dataset.map(preprocess_function)

  0%|          | 0/25500 [00:00<?, ?ex/s]

  0%|          | 0/4500 [00:00<?, ?ex/s]

In [ ]:
tokenized_train[0]

{'label': 1,
 'text': 'Te (Extroverted Thinking) (40%)  your valuation of / adherence to logic of external systems / hierarchies / methods Ti (Introverted Thinking) (48%)  your valuation of / adherence to your own...',
 'input_ids': [101,
  8915,
  1006,
  4654,
  13181,
  26686,
  3241,
  1007,
  1006,
  2871,
  1003,
  1007,
  2115,
  26004,
  1997,
  1013,
  29235,
  2000,
  7961,
  1997,
  6327,
  3001,
  1013,
  7632,
  6906,
  11140,
  3111,
  1013,
  4725,
  14841,
  1006,
  17174,
  26686,
  3241,
  1007,
  1006,
  4466,
  1003,
  1007,
  2115,
  26004,
  1997,
  1013,
  29235,
  2000,
  2115,
  2219,
  1012,
  1012,
  1012,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

In [ ]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
training_args = TrainingArguments(
    
    output_dir= ourpur_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25500
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7970
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster 

Step,Training Loss
500,0.654400
1000,0.631600
1500,0.625600
2000,0.594300
2500,0.579700
3000,0.566900
3500,0.507400
4000,0.458000
4500,0.447500
5000,0.400900


Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT/checkpoint-500
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT/checkpoint-500/special_tokens_map.json
Saving model checkpoint to /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT/checkpoint-1000
Configuration saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/NLP_Workspace/Data/BERT_data/TvsF_BERT/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/NLP_Workspace/D

TrainOutput(global_step=7970, training_loss=0.44819841755827516, metrics={'train_runtime': 937.2597, 'train_samples_per_second': 136.035, 'train_steps_per_second': 8.504, 'total_flos': 1970543626344576.0, 'train_loss': 0.44819841755827516, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4500
  Batch size = 16


<ipython-input-20-b1430cee0b64>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 1.1597352027893066,
 'eval_accuracy': 0.6137777777777778,
 'eval_f1': 0.6622619510299261,
 'eval_runtime': 9.7039,
 'eval_samples_per_second': 463.73,
 'eval_steps_per_second': 29.06,
 'epoch': 5.0}